In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
from __future__ import print_function, division
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


%matplotlib inline

In [ ]:
ROOT_DIR = './GTSRB/Final_Training/Images/'
TEST_DIR = './GTSRB/Final_Test/Images/'

In [ ]:
# pd.read_csv(glob('%s/GT-final_test.csv'%TEST_DIR)[0], sep=';')

## Define Dataset  (need to be adapted depending on dataset)

In [ ]:
class TrafficSignDataset(Dataset):
    """Traffic Signs Dataset"""

    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        
        if 'Train' in root_dir:
            folders = glob('%s/*'%root_dir)
            
            files = [glob(os.path.join(f, '*.ppm')) for f in folders]
            samp_num = [len(f) for f in files]
            
            max_num = max(samp_num)
            
            frame = pd.DataFrame()
            for f, folder, num in zip(files, folders, samp_num):
                tmp = pd.DataFrame({'Filename':f, 'ClassId': int(folder[-5:])})
                frame = frame.append(tmp) 
                frame = frame.append(tmp.sample(max_num-num, replace=True))
                
            
            self.frame = frame.sample(frac=1).reset_index(drop=True)
            self.transform = transform
            
            
        if 'Test' in root_dir:
            self.frame = pd.read_csv('%s/GT-final_test.csv'%TEST_DIR, sep=';')[['Filename', 'ClassId']]
            self.frame['Filename'] = self.frame['Filename'].apply(lambda x : os.path.join(root_dir, x))
            

    def __len__(self):
        return self.frame.shape[0]

    def __getitem__(self, idx):
        img_name = self.frame.loc[idx, 'Filename']
        image = io.imread(img_name)
        label = self.frame.loc[idx, 'ClassId']
        sample = {'image': image, 'label': torch.tensor(int(label)).to('cuda')}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
            
        return sample
    
    
traffic_data = TrafficSignDataset(ROOT_DIR)

In [ ]:
for i in range(5, 30, 5):
    plt.imshow(traffic_data[i]['image'])
    plt.show()

## Transformations & Augmentations

In [ ]:
from torchvision.transforms import RandomResizedCrop, RandomRotation, RandomAffine, ToPILImage, ToTensor, Resize, Grayscale

class ToDeviceTensor(object):
    def __call__(self, image_tensor):
        return ToTensor()(image_tensor).to('cuda')
    
    
augment_tf = [] # Addd some augmentations here

# Train
train_composed = transforms.Compose([ToPILImage()] + augment_tf + [Resize((32, 32)), ToDeviceTensor()])  
traffic_data_train = TrafficSignDataset(ROOT_DIR, transform=train_composed)
trainloader = DataLoader(traffic_data_train, batch_size=16)

# Test
test_composed = transforms.Compose([ToPILImage(), Resize((32, 32)), ToDeviceTensor()])
traffic_data_test = TrafficSignDataset(TEST_DIR, transform=test_composed)
testloader = DataLoader(traffic_data_test, batch_size=16)

show_composed = transforms.Compose([ToPILImage()] + augment_tf + [Resize((32, 32))]) 
traffic_data_show = TrafficSignDataset(ROOT_DIR, transform=show_composed)
for i in range(5, 30, 5):
    plt.imshow(traffic_data_show[i]['image'])
    plt.show()

## The Net

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

num_classes = 43

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 5) 
        self.conv2 = nn.Conv2d(16, 32, 5) 
        self.conv3 = nn.Conv2d(32, 128, 5) 
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.fc5 = nn.Linear(256*3*3, num_classes) 
        
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(0.5)

    def forward(self, x):
        # one activated conv layer
        x = F.relu(self.conv1(x)) # 32 -> 28
        x = self.pool(F.relu(self.conv2(x))) # 28 -> 24 -> 12
        x = self.drop(x)
        x = F.relu(self.conv3(x)) # 12 -> 8
        x = self.pool(F.relu(self.conv4(x))) # 8 -> 6 -> 3
        x = self.drop(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc5(x))
        x = F.log_softmax(x, dim=1)
        # final output
        return x

# instantiate and print your Net
net = Net().cuda()
print(net)

## Loss and Trainning

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.96)

In [ ]:
for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data['image'], data['label']

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

## Testing

In [ ]:
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))
with torch.no_grad():
    for data in testloader:
        images, labels = data['image'], data['label']
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        
        for i in range(16):
            try:
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            except:
                continue


for i in range(43):
    print('Accuracy of %5s : %2d %%' % (
        i, 100 * class_correct[i] / class_total[i]))

In [ ]:
sum(class_correct)/sum(class_total)